（※このファイルはGoogle Colaboratory用です。他の環境では未検証です）

（※**Googleドライブの機能を使用します。**途中の結果がドライブ上に保存されます。空き容量に注意してください）

（※2-1.以降の実行後、再度1-1.から実行したい場合は「ランタイム→すべてのランタイムのリセット...」を実行してリセットしてください）

##1. 音源の学習処理

1-1. 以下のコードを実行し、音声ファイルを圧縮したzipファイルをアップロードしてください。

In [0]:
%cd /content

# 音声の入ったzipファイルをアップロード
from google.colab import files
import os

uploaded = files.upload()

for fn in uploaded.keys():
    os.rename(fn, 'targets.zip')

1-2. メイン処理です。以下のコードを実行してください。

（途中でURLが表示され、止まることがあります。その場合はURLにアクセスし、キーを入力欄に貼り付けてください）

（タイムアウトなどで途中で終了した場合でも、このコードを再度実行すれば続きから再開されます）

（一度、他の音源を作成した場合は、実行前にドライブ上のnvcディレクトリを移動させるか、retrain_flagをFalseにしてください）

（学習処理は自動で終了します。Epoch: 1/100000とか出ますが本当に100000回はかかりません）

In [0]:
# ↓自動で音声ファイルを分割するかどうか
# （【数秒程度に分割していない場合はTrueにしてください】）
cut_flag = False
# ↓データの水増しを行うかどうか
# （精度を優先させたい場合はTrueにしてください）
increase_flag = False
# ↓途中で終了した場合にそこから再開するか。新規に作成する場合はFalseにすること
# （通常は変更する必要はありません）
retrain_flag = True
# ---

import zipfile
import os
import shutil
import glob

%cd /content

if not os.path.isdir('NVC_train'):
    # リポジトリをクローンし、フォルダ内へ移動
    !git clone --depth 1 https://github.com/NON906/NVC_train.git
    %cd NVC_train

    # Googleドライブに接続
    from google.colab import drive
    drive.mount('drive')
else:
    %cd NVC_train
    
!pip install tensorflow==1.13.1 > /dev/null 2>&1
import tensorflow as tf
    
# メイン処理開始

outputs_dir = os.getcwd() + '/drive/My Drive/nvc'

if not retrain_flag:
    if os.path.isdir(outputs_dir):
        shutil.rmtree(outputs_dir)
    if os.path.isdir('gen_targets'):
        shutil.rmtree('gen_targets')

if (not os.path.isfile(outputs_dir + '/target.nvm') or not os.path.isfile(outputs_dir + '/gen_targets.zip')) and not os.path.isdir(outputs_dir + '/targets'):
    # インストール
    !apt-get update > /dev/null 2>&1
    !apt-get -y install unar sox libsox-fmt-all csh > /dev/null 2>&1
    !pip install pyworld > /dev/null 2>&1

    # SPTKのインストール
    !wget http://downloads.sourceforge.net/sp-tk/SPTK-3.11.tar.gz > /dev/null 2>&1
    !unar SPTK-3.11.tar.gz > /dev/null 2>&1
    %cd SPTK-3.11
    !./configure > /dev/null 2>&1
    !make > /dev/null 2>&1
    !make install > /dev/null 2>&1
    %cd ..

    if cut_flag:
        # アップロードしたzipファイルを解凍
        !unar -o targets_zip ../targets.zip > /dev/null 2>&1

        if os.path.isdir('targets') == False:
            os.mkdir('targets')
        input_voices = glob.glob('targets_zip/**/*', recursive=True)
        del_files = glob.glob('targets_zip/**/.*', recursive=True)
        for input_voice in input_voices:
            if os.path.isdir(input_voice):
                del_files.append(input_voice)
        for del_file in del_files:
            input_voices.remove(del_file)
        for voice in input_voices:
            save_file =  voice[len('targets_zip/'):].replace('/', '__')
            # 音声ファイルの分割
            save_file = save_file.replace('.', '__')
            !sox "{voice}" "targets/{save_file}_.wav" silence -l 1 0.5 0.01% 1 0.5 0.01%: newfile : restart > /dev/null 2>&1
            !rm "{voice}" > /dev/null 2>&1
    else:
        !unar -o targets ../targets.zip > /dev/null 2>&1

from scripts.make_nvm import make_nvm_main
from scripts.gen_targets import gen_targets_main

if not os.path.isfile(outputs_dir + '/target.nvm'):
    make_nvm_main('targets', outputs_dir + '/target.nvm')
    
if not os.path.isfile(outputs_dir + '/gen_targets.zip'):
    if increase_flag:
        gen_targets_main('targets', 'gen_targets', outputs_dir + '/gen_targets.zip')
    else:
        gen_targets_main('targets', 'gen_targets', outputs_dir + '/gen_targets.zip', cut_loop_length=1, reverse_flag=False)
else:
    with zipfile.ZipFile(outputs_dir + '/gen_targets.zip') as existing_zip:
        existing_zip.extractall('gen_targets')
    
from scripts.target_train_tpu import target_train_tpu_main
from scripts.target_train_tpu_convert import target_train_tpu_convert_main
from tensorflow.keras.optimizers import Adam

if not os.path.isfile(outputs_dir + '/target_tpu.h5'):
    target_train_tpu_main('gen_targets', outputs_dir + '/target_tpu.h5', 5, 3200, 160, 1, optimizer=Adam(amsgrad=True), epochs=100000)
else:
    target_train_tpu_main('gen_targets', outputs_dir + '/target_tpu.h5', 5, 3200, 160, 1, optimizer=Adam(amsgrad=True), epochs=100000, retrain_file=outputs_dir + '/target_tpu.h5')
target_train_tpu_convert_main(outputs_dir + '/target_tpu.h5', outputs_dir + '/target_weight.h5', outputs_dir + '/target.json')

print('完了しました')


---

##2. 学習データの変換

2-1. 以下のコードを実行してください。

In [0]:
!pip install tensorflow==1.7.1

2-2. 上部メニューの「ランタイム→ランタイムを再起動...」から再起動してください。

2-3. 以下のコードを実行してください。

In [0]:
import sys
import os

%cd /content

if not os.path.isdir('NVC_train'):
    # リポジトリをクローンし、フォルダ内へ移動
    !git clone --depth 1 https://github.com/NON906/NVC_train.git
    %cd NVC_train

    # Googleドライブに接続
    from google.colab import drive
    drive.mount('drive')
else:
    %cd NVC_train
    
from scripts.target_convert_h5_to_pb import convert_h5_to_pb_main
from scripts.make_nvz import make_nvz_main

outputs_dir = os.getcwd() + '/drive/My Drive/nvc'

convert_h5_to_pb_main(outputs_dir + '/target_weight.h5', outputs_dir + '/target.json', outputs_dir + '/target.pb', 'target_dense_f/add')
make_nvz_main(outputs_dir + '/target.nvz', outputs_dir + '/target.nvm', outputs_dir + '/target.pb')

2-4. 2-3.の実行後、以下のコードからダウンロード版用のファイルをダウンロードできます。

In [0]:
%cd /content/NVC_train

from google.colab import files
import os

outputs_dir = os.getcwd() + '/drive/My Drive/nvc'
files.download(outputs_dir + '/target.nvz')